# Merging, converting and filtering variant data using PLINK 2.0

> This notebook shows how to interact with genomic data in bed/bim/bam format using PLINK 2.0. We will learn how to convert between PLINK 1.x and PLINK 2.x file formats, merge variants from different chromosomes into a single file and filter them based on variant completeness and minor allelic frequencies (MAF). While runtime is approximately 12 hours it can be significantly scaled down (see subsequent notes).

- runtime: 12 hours
- recommended instance: mem1_ssd1_v2_x16
- estimated cost: ca. £5.00

This notebook depends on:
* **PLINK install**

This process filters and then merges all the chromosomes into one file. More filters, prior to merging will likely speed this process.

## List the exome sequences data directories in your project

Please note, that depending on your project's MTA the list of files might differ.

In [1]:
ls /mnt/project/Bulk/'Exome sequences'/

'Exome OQFE CRAM files'
'Exome OQFE variant call files (VCFs)'
'Population level exome OQFE variants, BGEN format - final release'
'Population level exome OQFE variants, BGEN format - interim 450k release'
'Population level exome OQFE variants, PLINK format - final release'
'Population level exome OQFE variants, PLINK format - interim 450k release'
'Population level exome OQFE variants, pVCF format - final release'
'Population level exome OQFE variants, pVCF format - interim 450k release'


## List the population variant files in PLINK 1.x (bed/bim/fam) format

In [2]:
ls /mnt/project/Bulk/'Exome sequences'/'Population level exome OQFE variants, PLINK format - final release'/

helper_files            ukb23158_c18_b0_v1.bed  ukb23158_c4_b0_v1.bim
ukb23158_c10_b0_v1.bed  ukb23158_c18_b0_v1.bim  ukb23158_c4_b0_v1.fam
ukb23158_c10_b0_v1.bim  ukb23158_c18_b0_v1.fam  ukb23158_c5_b0_v1.bed
ukb23158_c10_b0_v1.fam  ukb23158_c19_b0_v1.bed  ukb23158_c5_b0_v1.bim
ukb23158_c11_b0_v1.bed  ukb23158_c19_b0_v1.bim  ukb23158_c5_b0_v1.fam
ukb23158_c11_b0_v1.bim  ukb23158_c19_b0_v1.fam  ukb23158_c6_b0_v1.bed
ukb23158_c11_b0_v1.fam  ukb23158_c1_b0_v1.bed   ukb23158_c6_b0_v1.bim
ukb23158_c12_b0_v1.bed  ukb23158_c1_b0_v1.bim   ukb23158_c6_b0_v1.fam
ukb23158_c12_b0_v1.bim  ukb23158_c1_b0_v1.fam   ukb23158_c7_b0_v1.bed
ukb23158_c12_b0_v1.fam  ukb23158_c20_b0_v1.bed  ukb23158_c7_b0_v1.bim
ukb23158_c13_b0_v1.bed  ukb23158_c20_b0_v1.bim  ukb23158_c7_b0_v1.fam
ukb23158_c13_b0_v1.bim  ukb23158_c20_b0_v1.fam  ukb23158_c8_b0_v1.bed
ukb23158_c13_b0_v1.fam  ukb23158_c21_b0_v1.bed  ukb23158_c8_b0_v1.bim
ukb23158_c14_b0_v1.bed  ukb23158_c21_b0_v1.bim  ukb23158_c8_b0_v1.fam
ukb23158_c14_b0_v1.b

### Install and test the PLINK2 binary
#### We recommend installing plink using the links available here:
https://www.cog-genomics.org/plink/2.0/

#### You can download the binary (i.e. AVX2 Intel; for example, using `wget <URL>`), before unzipping (`unzip <zip file>`) then making it exectutable (`chmod a+x <name>`)

#### if preferred, Plink is also available in the following locations:
https://anaconda.org/bioconda/plink2; https://github.com/chrchang/plink-ng

#### Once installed, continue with the below code chunks.

In [2]:
#test plink works
./plink2 --version

PLINK v2.00a6LM AVX2 Intel (27 Sep 2023)


## Prepare the list of files to that will be filtered
#### **NB for this demo we use the first 8 chromosomes**
#### **this will take a few hours to generate the output files**
#### **subsequent notebooks could instead be run on fewer inputs - e.g., chromosomes 20-22 (i.e. the three smallest chromosomes)**
#### **However, a couple of downstream notebook analyses may not show significant results:**
#### **this may not be essential if you are simply aiming to observe how the codes work and therefore may wish to use chrs 20-22**


In [ ]:
rm -f list_beds.txt
for chr in {1..8}; do # **OPTIONAL CHANGE TO** {20..22} 
    echo "ukb23158_c${chr}_b0_v1" >> list_beds.txt; 
done

The list is a white-space delimited file listing a bed, bim and fam files in each column:


## Apply filters to each file in the list and export as PLINK 1.x dataset (bed/bim/fam)

List of filters:

- `--maf` filters out all variants with allele frequency below 0.01, different thresholds can be  provided by modifying a numeric value after this parameter
- `--mac` impose a filter on lower allele count, only variants with at least 100 cases will be included
- `--geno` filters out all variants with missing call rates exceeding the provided value - setting this value to 0 removes all incomplete variants
- `--hwe` filters out all variants which have Hardy-Weinberg equilibrium exact test p-value below the provided threshold
- `--mind` filter out samples with missing calls above treshold of 10%
- `--no-psam-pheno` ignore all phenotype data in the sample information file and allows .fam files with no phenotype column to be loaded (such as withdrawn participants) 

In [ ]:
for x in $(cat list_beds.txt);do \
./plink2 \
  --bfile /mnt/project/Bulk/'Exome sequences'/'Population level exome OQFE variants, PLINK format - final release'/$x \
  --maf 0.01 --mac 100 --geno 0 --hwe 1e-15 \
  --no-psam-pheno \
  --mind 0.1 \
  --write-snplist --write-samples --no-id-header \
  --make-bed \
  --out ${x}_filt; done

## Make a suitably formatted list ("filtered_list2.txt") of all the filtered files

In [5]:
ls *filt.bed | sed 's/.bed//g' > filtered_list.txt
awk '{print $1".bed", $1".bim", $1".fam"}' filtered_list.txt > filtered_list2.txt

## Merge genomic files to a single PLINK 1.x 

Change `--make-bed` to `--make-pgen` to produce PLINK 2.x formated files (pgen/pvar/psam) they are faster to work with and have significntly smaller size

In [ ]:
./plink2 \
  --no-pheno \
  --pmerge-list filtered_list2.txt \
  --make-bed \
  --out maf_flt_8chroms

## Validate the output dataset

In [ ]:
./plink2 \
  --pfile maf_flt_8chroms \
  --validate

## Upload the completed dataset to your project

We will use this dataset in other notebooks

In [ ]:
dx mkdir bed_maf
dx upload maf_flt_8chroms* --path bed_maf/